In [1]:
import torch
import numpy as np
import gym

env = gym.make("CartPole-v1")

In [2]:
def _tilings_features(env, n_tilings, bin_sizes, offsets):

    tilings = []

    upper_bounds = env.observation_space.high
    lower_bounds = env.observation_space.low
    n_features = env.observation_space.shape[0]

    for i in range(n_tilings):
        bin = bin_sizes[i]
        offset = offsets[i]
        tiling = []

        for j in range(n_features):
            f_tiling = np.linspace(lower_bounds[i], upper_bounds[i], bin[j]+1)[1:-1] + offset[j]
            tiling.append(f_tiling)
        
        tilings.append(tiling)

    return np.array(tilings)


In [9]:
x=torch.Tensor([0.02231883,  0.19944809, -0.02742709, -0.31805503])
print(x)
print(torch.cat([x ** i for i in range(1, 3)],-1))

tensor([ 0.0223,  0.1994, -0.0274, -0.3181])
tensor([ 0.0223,  0.1994, -0.0274, -0.3181,  0.0005,  0.0398,  0.0008,  0.1012])


In [4]:
print(env.observation_space.high)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


In [24]:
custom_tilings = [[-3,-1,1,3],[-5,-1,1,5],[-0.3,-0.1,-0.01,0.01,0.1,0.3],[-5,-1,1,5]]
n_tilings = 5**4
print(n_tilings)

625


In [25]:
x=torch.Tensor([5,  0.19944809, 0.000742709, -0.31805503])

for i, num in enumerate(x):
    print(np.digitize(num,custom_tilings[i]))



4
2
3
2


In [99]:
def _tiling_features(x):
    tiling_coords = [-0.3,-0.1,-0.01,0.01,0.1,0.3]
    
    
    try:
        f_tiling = np.digitize(x[:,2], tiling_coords)
        
        new_features = np.zeros((x.shape[0],len(tiling_coords)+1))
        new_features[np.arange(x.shape[0]),f_tiling]=1
        
        return torch.cat([x[:,[0,1,3]],torch.Tensor(new_features)], -1)

    except:
        f_tiling = np.digitize(x[2], tiling_coords)
        
        new_features = np.zeros(len(tiling_coords)+1)
        new_features[f_tiling]=1
        
        return torch.cat([x[[0,1,3]],torch.Tensor(new_features)], -1)
    
    

def _polynomial_features(x):
    return torch.cat([x ** i for i in range(1, 3)], -1)



In [100]:
x=torch.Tensor([[5,  0.19944809, 0.02742709, -0.31805503], [5,  0.19944809, 0.000742709, -0.31805503]])
print(_polynomial_features(x))
print(_tiling_features(x))

tensor([[ 5.0000e+00,  1.9945e-01,  2.7427e-02, -3.1806e-01,  2.5000e+01,
          3.9780e-02,  7.5225e-04,  1.0116e-01],
        [ 5.0000e+00,  1.9945e-01,  7.4271e-04, -3.1806e-01,  2.5000e+01,
          3.9780e-02,  5.5162e-07,  1.0116e-01]])
tensor([[ 5.0000e+00,  1.9945e-01,  2.7427e-02, -3.1806e-01,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 5.0000e+00,  1.9945e-01,  7.4271e-04, -3.1806e-01,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00]])


In [87]:
x=torch.Tensor([5,  0.19944809, 0.02742709, -0.31805503])
x1=torch.Tensor([[5,  0.19944809, 0.02742709, -0.31805503], [5,  0.19944809, 0.000742709, -0.31805503]])


In [90]:
x1.shape

torch.Size([2, 4])